In [3]:
#%%
import os
import sys
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import openpyxl
import pandas as pd
import math
import itertools
import matplotlib.patches as patches
import seaborn as sns
from decimal import Decimal
import glob
import csv
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn.svm 
import sklearn.metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras import layers
import pickle

def get_app_list(fileName):
    """get metrics list

    Arguments:
        fileName {string} -- the file name of application file with absolute path

    Returns:
        app {list} -- the app  list
    """
    apps = []
    with open(fileName) as f:
        for line in f.readlines():
            if not line.startswith('#'):
                words = line.strip().split(',')
                app = words[0].strip()
                app_num = words[1].strip()
                apps.append(app+app_num)
        f.close()
    return apps

def load_file(filepath):
    
    dataframe = pd.read_csv(filepath, index_col=0)
    if (dataframe.shape[1] == 4 and dataframe.shape[0] > max_len):
        #if dataframe["u_gpu"].sum() == 0:
        #    #os.remove(filepath)
        #    print("1", filepath)
        #else:
            return dataframe.values[:max_len,:]
    else:
        print("2", filepath)
        
def load_group(arch):

    y_label = []
    app_label = []
    data_group = []
    category = "mybench"
    pathfolder = '/home/pzou/projects/Power_Signature/results-50/%s/%s/power-combine'%(category, arch)
    app_list = app_list = get_app_list("/home/pzou/projects/Power_Signature/Scripts/applications_%s.csv"%(category))
    for app in app_list:
        fileName =app+"_res.csv"
        data = load_file(os.path.join(pathfolder, fileName))
        data_group.append(data)
        y_label.append(0)
    print("normal count", len(data_group))
    
    normal_count =len(data_group)
    category = "risky"
    pathfolder = '/home/pzou/projects/Power_Signature/results-50/%s/%s/power-combine'%(category, arch)
    app_list = app_list = get_app_list("/home/pzou/projects/Power_Signature/Scripts/applications_%s.csv"%(category))
    for app in app_list:
        fileName =app+"_res.csv"
        data = load_file(os.path.join(pathfolder, fileName))
        data_group.append(data)       
        y_label.append(1)
        
    print("risk count", len(data_group)-normal_count)
    data_group = np.asarray(data_group)    
    return data_group, y_label





                


print("start loading")    
arch="p100"
max_len = 1800
data_group, y_label = load_group(arch)
y_label = pd.Series(y_label)

train_index = [652, 267, 779, 633, 465, 317, 108, 704, 768, 181, 627, 813, 814, 160, 508, 84, 452, 90, 309, 808, 311, 673, 411, 42, 713, 773, 394, 676, 157, 755, 829, 275, 664, 412, 391, 595, 111, 612, 459, 803, 458, 610, 557, 802, 55, 677, 444, 342, 40, 753, 582, 288, 615, 178, 176, 597, 58, 340, 16, 291, 734, 751, 818, 480, 707, 387, 455, 549, 786, 29, 821, 355, 577, 318, 694, 203, 32, 513, 372, 156, 530, 402, 526, 718, 519, 3, 522, 130, 540, 721, 737, 447, 417, 378, 113, 48, 289, 445, 206, 593, 285, 642, 225, 503, 120, 407, 624, 221, 432, 171, 666, 756, 591, 572, 659, 698, 123, 335, 180, 161, 608, 542, 117, 828, 488, 580, 396, 468, 240, 134, 155, 139, 834, 60, 518, 165, 300, 784, 399, 297, 131, 314, 783, 757, 629, 471, 38, 249, 53, 194, 136, 502, 579, 281, 35, 440, 536, 798, 66, 754, 400, 576, 52, 377, 616, 521, 825, 239, 246, 512, 28, 710, 265, 559, 244, 103, 276, 393, 110, 456, 159, 235, 427, 112, 470, 744, 473, 598, 329, 262, 223, 853, 429, 728, 438, 817, 167, 644, 241, 693, 711, 815, 24, 294, 546, 780, 724, 150, 41, 346, 94, 457, 217, 660, 381, 23, 268, 388, 352, 416, 292, 200, 173, 100, 637, 91, 650, 804, 515, 646, 511, 222, 214, 188, 354, 514, 552, 571, 788, 353, 76, 242, 862, 73, 847, 810, 830, 425, 347, 460, 118, 643, 114, 611, 785, 5, 524, 504, 484, 69, 14, 854, 532, 369, 78, 202, 154, 640, 479, 684, 4, 578, 690, 54, 467, 266, 678, 293, 71, 566, 2, 25, 308, 216, 343, 749, 558, 166, 596, 601, 331, 177, 715, 375, 437, 247, 373, 832, 472, 499, 833, 525, 696, 307, 501, 234, 620, 430, 543, 191, 151, 435, 256, 662, 575, 747, 383, 145, 695, 761, 533, 215, 748, 774, 738, 196, 727, 495, 17, 668, 27, 277, 395, 228, 320, 147, 816, 494, 449, 617, 106, 350, 523, 389, 448, 93, 259, 554, 763, 170, 254, 252, 306, 547, 687, 819, 428, 186, 269, 162, 413, 667, 771, 98, 6, 692, 735, 229, 264, 368, 600, 742, 729, 770, 403, 841, 189, 332, 303, 260, 144, 283, 800, 11, 709, 594, 531, 469, 127, 632, 57, 33, 237, 791, 282, 586, 362, 243, 603, 92, 210, 337, 301, 85, 190, 39, 500, 507, 516, 104, 795, 233, 105, 341, 253, 408, 422, 208, 843, 663, 82, 497, 789, 583, 672, 87, 64, 270, 605, 274, 683, 767, 153, 613, 852, 410, 517, 44, 625, 390, 220, 140, 349, 357, 133, 261, 278, 79, 838, 115, 705, 722, 380, 801, 184, 681, 358, 36, 339, 723, 420, 149, 9, 45, 95, 776, 858, 548, 560, 298, 290, 415, 313, 421, 657, 122, 726, 706, 361, 336, 135, 733, 651, 367, 827, 778, 619, 8, 97, 835, 348, 509, 822, 703, 21, 482, 846, 405, 850, 731, 478, 590, 491, 46, 364, 296, 49, 487, 224, 219, 811, 851, 647, 201, 562, 622, 855, 63, 604, 245, 442, 752, 574, 0, 539, 689, 187, 406, 148, 26, 461, 670, 655, 59, 714, 326, 424, 720, 477, 43, 661, 635, 528, 736, 483, 218, 741, 857, 792, 609, 485, 125, 777, 61, 81, 680, 324, 385, 856, 56, 19, 824, 86, 758, 649, 316, 34, 138, 766, 588, 498, 538, 555, 96, 83, 446, 474, 764, 366, 697, 585, 840, 760, 401, 759, 392, 806, 745, 271, 506, 860, 434, 648, 807, 561, 207, 319, 844, 132, 50, 238, 564, 592, 450, 31, 121, 211, 351, 454, 302, 708, 88, 325, 323, 255, 570, 545, 550, 359, 51, 772, 102, 195, 679, 805, 628, 124, 404, 730, 287, 831, 47, 725, 587, 607, 433, 674, 263, 360, 75, 466, 299, 534, 584, 493, 527, 168]
test_index = [286, 169, 363, 258, 74, 589, 732, 273, 284, 439, 70, 251, 146, 423, 15, 374, 581, 630, 750, 797, 419, 68, 510, 626, 743, 99, 379, 204, 535, 492, 322, 158, 837, 414, 464, 848, 37, 365, 859, 344, 426, 839, 682, 849, 544, 762, 143, 823, 193, 812, 175, 675, 481, 463, 197, 631, 152, 573, 257, 272, 236, 556, 65, 20, 529, 599, 328, 67, 476, 248, 443, 520, 230, 182, 765, 205, 686, 280, 164, 685, 141, 231, 826, 638, 436, 775, 198, 739, 7, 185, 183, 568, 101, 654, 740, 489, 305, 128, 327, 793, 475, 746, 656, 451, 321, 398, 209, 861, 137, 386, 769, 109, 781, 820, 142, 192, 719, 541, 334, 232, 279, 441, 315, 639, 453, 653, 356, 665, 213, 62, 18, 614, 565, 717, 641, 799, 22, 490, 310, 330, 712, 551, 227, 658, 107, 809, 172, 702, 634, 382, 89, 80, 333, 842, 537, 30, 10, 836, 701, 602, 787, 796, 77, 409, 1, 345, 376, 212, 794, 486, 370, 116, 397, 431, 179, 174, 563, 688, 553, 669, 790, 119, 623, 671, 295, 13, 782, 699, 462, 250, 845, 338, 418, 129, 226, 199, 126, 384, 163, 567, 700, 606, 304, 569, 505, 621, 636, 312, 371, 72, 496, 645, 618, 716, 691, 12]


#y_train, y_test = train_test_split(y_label, test_size=0.25, random_state=2)
train_index = y_train.index.tolist()
test_index = y_test.index.tolist()
print("Done loading")
#%%

start loading
normal count 694
risk count 169
Done loading


In [4]:
#%%
print(train_index)
print(test_index)
#%%

[652, 267, 779, 633, 465, 317, 108, 704, 768, 181, 627, 813, 814, 160, 508, 84, 452, 90, 309, 808, 311, 673, 411, 42, 713, 773, 394, 676, 157, 755, 829, 275, 664, 412, 391, 595, 111, 612, 459, 803, 458, 610, 557, 802, 55, 677, 444, 342, 40, 753, 582, 288, 615, 178, 176, 597, 58, 340, 16, 291, 734, 751, 818, 480, 707, 387, 455, 549, 786, 29, 821, 355, 577, 318, 694, 203, 32, 513, 372, 156, 530, 402, 526, 718, 519, 3, 522, 130, 540, 721, 737, 447, 417, 378, 113, 48, 289, 445, 206, 593, 285, 642, 225, 503, 120, 407, 624, 221, 432, 171, 666, 756, 591, 572, 659, 698, 123, 335, 180, 161, 608, 542, 117, 828, 488, 580, 396, 468, 240, 134, 155, 139, 834, 60, 518, 165, 300, 784, 399, 297, 131, 314, 783, 757, 629, 471, 38, 249, 53, 194, 136, 502, 579, 281, 35, 440, 536, 798, 66, 754, 400, 576, 52, 377, 616, 521, 825, 239, 246, 512, 28, 710, 265, 559, 244, 103, 276, 393, 110, 456, 159, 235, 427, 112, 470, 744, 473, 598, 329, 262, 223, 853, 429, 728, 438, 817, 167, 644, 241, 693, 711, 815, 24, 294,

In [1]:
#%%
def rnnClassify(data_group, y_label,  arch):
    #%%
    y_label = pd.Series(y_label)
    X_train, X_test, y_train, y_test = train_test_split(data_group, y_label, test_size=0.25, random_state=1)
    #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
    #print(X_train)
    X_train = np.asarray(X_train)
    X_test = np.asarray(X_test)
    n_timesteps = max_len
    n_features = 4
    model = tf.keras.Sequential()
    model.add(layers.LSTM(256, input_shape=(n_timesteps, n_features)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
                
    
    checkpoint_path = "%s/resourceAll-50-%s.hdf5"%(arch,arch)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                 save_best_only=True,
                                                 monitor='val_loss', 
                                                 mode='min')

    hist= model.fit(x=X_train,y=y_train,
            epochs=120,
            validation_split=0.25,
            callbacks = [cp_callback],
            batch_size = 256,
            class_weight={1:5, 0:1}
             )

    df = pd.DataFrame.from_dict(hist.history)
    df.to_csv("%s/resourceAll-50-%s-history.csv"%(arch,arch))
        
    model = tf.keras.models.load_model(checkpoint_path)        
    loss, accuracy = model.evaluate(x=X_test, y=y_test)    
    with open("%s/resourceAll-50-%s-testAccurcy.txt"%(arch,arch), "w+") as f:
        f.write(str(accuracy))
        f.close()
        
    y_predict = model.predict_classes(data_group)
    y_predict = [ i[0] for i in y_predict.tolist()]
    df = pd.DataFrame.from_dict({"y_index":y_label.index.tolist(), "predict":y_predict})
    df.to_csv("%s/resourceAll-50-%s-All.csv"%(arch,arch))
    
    
    y_predict = model.predict_classes(X_test)
    y_predict = [ i[0] for i in y_predict.tolist()]
    df = pd.DataFrame.from_dict({"y_index":y_test.index.tolist(), "predict":y_predict})
    df.to_csv("%s/resourceAll-50-%s-test.csv"%(arch,arch))

def rnnClassifyOne(data_group, y_label, arch):
    #%%
    y_label = pd.Series(y_label)
    col_list = ["power", "memSize", "gpuU", "memU"]
    for i in [0, 1, 2, 3]:       
        data_X = data_group[:, :, [i]]
        X_train, X_test, y_train, y_test = train_test_split(data_X, y_label, test_size=0.25, random_state=1)
        #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
        #print(X_train)
        X_train = np.asarray(X_train)
        X_test = np.asarray(X_test)
        n_timesteps = 3600
        n_features = 1
        model = tf.keras.Sequential()
        model.add(layers.LSTM(256, input_shape=(n_timesteps, n_features)))
        model.add(layers.Dropout(0.4))
        model.add(layers.Dense(128, activation='relu'))
        model.add(layers.Dense(16, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))

        model.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])
                    
        fileM = col_list[i]
        checkpoint_path = "%s/%s-%s.hdf5"%(arch,fileM ,arch)
        cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                     save_best_only=True,
                                                     monitor='val_loss', 
                                                     mode='min')

        hist= model.fit(x=X_train,y=y_train,
                epochs=120,
                validation_split=0.25,
                callbacks = [cp_callback],
                batch_size = 384,
                class_weight={1:5, 0:1}
                 )

        df = pd.DataFrame.from_dict(hist.history)
        df.to_csv("%s/%s-%s-history.csv"%(arch,fileM ,arch))
            
            
        loss, accuracy = model.evaluate(x=X_test, y=y_test)    
        with open("%s/%s-%s-testAccurcy.txt"%(arch,fileM ,arch), "w+") as f:
            f.write(str(accuracy))
            f.close()
            
        y_predict = model.predict_classes(data_X)
        y_predict = [ i[0] for i in y_predict.tolist()]
        df = pd.DataFrame.from_dict({"y_index":y_label.index.tolist(), "predict":y_predict})
        df.to_csv("%s/%s-%s-ALL.csv"%(arch,fileM ,arch))
        
        
        y_predict = model.predict_classes(X_test)
        y_predict = [ i[0] for i in y_predict.tolist()]
        df = pd.DataFrame.from_dict({"y_index":y_test.index.tolist(), "predict":y_predict})
        df.to_csv("%s/%s-%s-test.csv"%(arch,fileM ,arch))
    
def rnnClassifyTwo(data_group, y_label, arch):
    #%%
    y_label = pd.Series(y_label)
    col_list = ["power", "memSize", "gpuU", "memU"]
    for i in [0, 1, 2]:       
        for j in range(i+1, 4):        
            data_X = data_group[:, :, [i, j]]
            X_train, X_test, y_train, y_test = train_test_split(data_X, y_label, test_size=0.25, random_state=1)
            #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
            #print(X_train)
            X_train = np.asarray(X_train)
            X_test = np.asarray(X_test)
            n_timesteps = 1800
            n_features = 2
            model = tf.keras.Sequential()
            model.add(layers.LSTM(256, input_shape=(n_timesteps, n_features)))
            model.add(layers.Dropout(0.4))
            model.add(layers.Dense(128, activation='relu'))
            model.add(layers.Dense(16, activation='relu'))
            model.add(layers.Dense(1, activation='sigmoid'))

            model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])
                        
            fileM = col_list[i] + "-" + col_list[j]
            checkpoint_path = "%s/%s-%s.hdf5"%(arch,fileM ,arch)
            cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                         save_best_only=True,
                                                         monitor='val_loss', 
                                                         mode='min')

            hist= model.fit(x=X_train,y=y_train,
                    epochs=120,
                    validation_split=0.25,
                    callbacks = [cp_callback],
                    batch_size = 384,
                    class_weight={1:5, 0:1}
                     )

            df = pd.DataFrame.from_dict(hist.history)
            df.to_csv("%s/%s-%s-history.csv"%(arch,fileM ,arch))
                
                
            loss, accuracy = model.evaluate(x=X_test, y=y_test)    
            with open("%s/%s-%s-testAccurcy.txt"%(arch,fileM ,arch), "w+") as f:
                f.write(str(accuracy))
                f.close()
                
            y_predict = model.predict_classes(data_X)
            y_predict = [ i[0] for i in y_predict.tolist()]
            df = pd.DataFrame.from_dict({"y_index":y_label.index.tolist(), "predict":y_predict})
            df.to_csv("%s/%s-%s-ALL.csv"%(arch,fileM ,arch))
            
            
            y_predict = model.predict_classes(X_test)
            y_predict = [ i[0] for i in y_predict.tolist()]
            df = pd.DataFrame.from_dict({"y_index":y_test.index.tolist(), "predict":y_predict})
            df.to_csv("%s/%s-%s-test.csv"%(arch,fileM ,arch))
            
def rnnClassifyThree(data_group, y_label, arch):
    #%%
    y_label = pd.Series(y_label)
    col_list = ["power", "memSize", "gpuU", "memU"]
    for i in [0, 1]:       
        for j in range(i+1, 3):    
            for k in range(j+1, 4):    
                data_X = data_group[:,:, [i, j, k]]
                X_train, X_test, y_train, y_test = train_test_split(data_X, y_label, test_size=0.25, random_state=1)
                #X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
                #print(X_train)
                X_train = np.asarray(X_train)
                X_test = np.asarray(X_test)
                n_timesteps = 1800
                n_features = 3
                model = tf.keras.Sequential()
                model.add(layers.LSTM(256, input_shape=(n_timesteps, n_features)))
                model.add(layers.Dropout(0.4))
                model.add(layers.Dense(128, activation='relu'))
                model.add(layers.Dense(16, activation='relu'))
                model.add(layers.Dense(1, activation='sigmoid'))

                model.compile(optimizer='adam',
                            loss='binary_crossentropy',
                            metrics=['accuracy'])
                            
                fileM = col_list[i] + "-" + col_list[j]+ "-" + col_list[k]
                checkpoint_path = "%s/%s-%s.hdf5"%(arch,fileM ,arch)
                cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, 
                                                             save_best_only=True,
                                                             monitor='val_loss', 
                                                             mode='min')

                hist= model.fit(x=X_train,y=y_train,
                        epochs=120,
                        validation_split=0.25,
                        callbacks = [cp_callback],
                        batch_size = 384,
                        class_weight={1:5, 0:1}
                         )

                df = pd.DataFrame.from_dict(hist.history)
                df.to_csv("%s/%s-%s-history.csv"%(arch,fileM ,arch))
                    
                model = tf.keras.models.load_model(checkpoint_path)    
                loss, accuracy = model.evaluate(x=X_test, y=y_test)    
                with open("%s/%s-%s-testAccurcy.txt"%(arch,fileM ,arch), "w+") as f:
                    f.write(str(accuracy))
                    f.close()
                    
                y_predict = model.predict_classes(data_X)
                y_predict = [ i[0] for i in y_predict.tolist()]
                df = pd.DataFrame.from_dict({"y_index":y_label.index.tolist(), "predict":y_predict})
                df.to_csv("%s/%s-%s-ALL.csv"%(arch,fileM ,arch))
                
                
                y_predict = model.predict_classes(X_test)
                y_predict = [ i[0] for i in y_predict.tolist()]
                df = pd.DataFrame.from_dict({"y_index":y_test.index.tolist(), "predict":y_predict})
                df.to_csv("%s/%s-%s-test.csv"%(arch,fileM ,arch))
                

rnnClassify(data_group, y_label,  arch)
    #rnnClassifyOne(data_group, y_label, arch)
    #rnnClassifyTwo(data_group, y_label, arch)
    #rnnClassifyThree(data_group, y_label, arch)
    
    

#%%

NameError: name 'data_group' is not defined